# Unitary Decomposition Based on JanusQ-CT

**Author:** Congliang Lang \& Siwei Tan  

**Date:** 8/4/2024

Based on "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features (MICRO 2023][1]"

[1]: https://scholar.google.com/scholar_url?url=https://dl.acm.org/doi/abs/10.1145/3613424.3614274%3Fcasa_token%3DffjIB1hQ4ZwAAAAA:8MajDLrDOC74WoeMf7r7AoQ-koxCa4E1TNqQg3GSDz03xUX6XdE3toNTM-YdM_e4rKEusMceJ6BGJg&hl=zh-CN&sa=T&oi=gsb&ct=res&cd=0&d=11146218754516883150&ei=42YSZpPlFL6s6rQPtt6x6Ac&scisig=AFWwaeYaiu2hyx8HUJ_7Buf9Mwom

The expressiveness of gate vectors enables JanusQ-CT to reconstruct sub-circuits via gate vectors, which is a necessary ability for tasks involving circuit generation, such as the unitary decomposition. Unitary decomposition is a task that takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.

In [1]:
import sys
sys.path.append('..')
import os
os.chdir("..")
import logging
logging.basicConfig(level=logging.WARN)
import ray
ray.init(log_to_driver=False)
from qiskit import QuantumCircuit
from qiskit.quantum_info import random_unitary
from janusq.data_objects.circuit import qiskit_to_circuit
from janusq.data_objects.backend import  LinearBackend
from janusq.analysis.vectorization import RandomwalkModel
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.unitary_decompostion import decompose
import time

2024-04-16 11:28:17,442	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67059712 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-04-16 11:28:17,582	INFO worker.py:1724 -- Started a local Ray instance.


## Construct U2V model

The U2V model serves as the bridge between unitaries and gate vectors, where the sub-circuits reconstructed from these candidate vectors will replace the search space of QFAST. To build such a model, we obtain a U2V dataset composed of <$unitary,~\{vectors\}$> pairs, derived from a set of random circuits generated with the same scheme mentioned in Section~\ref{sec:equ_prediction}.

In [2]:
n_qubits = 4
backend = LinearBackend(n_qubits, 1, basis_two_gates = ['crz'])

In [3]:

from janusq.analysis.unitary_decompostion import U2VModel

n_step = 2

dataset = random_circuits(backend, n_circuits=50, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(
    n_step, 4 ** n_step, backend, directions=('parallel', 'next'))
vec_model.train(dataset, multi_process=True,
                        remove_redundancy=False)

u2v_model = U2VModel(vec_model)
data = u2v_model.construct_data(dataset, multi_process=False)
u2v_model.train(data, n_qubits)


100%|██████████| 6/6 [00:01<00:00,  5.41it/s]


len(Us) =  971 len(gate_vecs) =  971
Start construct U2VMdoel
Finish construct U2VMdoel, costing 1.0926015377044678s


## Decompose a unitary

In [4]:
# generate a random unitary
unitary = random_unitary(2**n_qubits).data

# apply decomposition
start_time = time.time()
quct_circuit = decompose(unitary, allowed_dist = 0.2, backend = backend, u2v_model = u2v_model)
quct_time = time.time() - start_time

str(quct_circuit), quct_time

In [ ]:
# compare it with the qsd method
from qiskit.synthesis.unitary.qsd import qs_decomposition

start_time =time.time()
qc = qs_decomposition(unitary)

qsd_circuit = qiskit_to_circuit(qc)
qsd_time = time.time() - start_time

str(qsd_circuit),qsd_time

([[{'name': 'circuit-941', 'qubits': [0, 1], 'params': []},
   {'name': 'u', 'qubits': [2], 'params': [0.0, 0.0, -0.4716580601924959]},
   {'name': 'u', 'qubits': [3], 'params': [0.0, 0.0, 0.4356579991759455]},
   {'name': 'u', 'qubits': [4], 'params': [0.0, 0.0, 0.1712784335716318]}],
  [{'name': 'cx', 'qubits': [0, 2], 'params': []}],
  [{'name': 'u', 'qubits': [2], 'params': [0.0, 0.0, -0.11186505751865095]}],
  [{'name': 'cx', 'qubits': [1, 2], 'params': []}],
  [{'name': 'u', 'qubits': [2], 'params': [0.0, 0.0, -0.6481220500555348]}],
  [{'name': 'cx', 'qubits': [0, 2], 'params': []}],
  [{'name': 'u', 'qubits': [2], 'params': [0.0, 0.0, -1.3497346726568913]}],
  [{'name': 'cx', 'qubits': [1, 2], 'params': []}],
  [{'name': 'u', 'qubits': [2], 'params': [1.611940250574522, 0.0, 0.0]},
   {'name': 'circuit-950', 'qubits': [0, 1], 'params': []}],
  [{'name': 'u',
    'qubits': [2],
    'params': [1.5707963267948966, 0.0, 3.141592653589793]}],
  [{'name': 'cx', 'qubits': [0, 2], 'par

In [ ]:
synthesis_method_result = [qsd_circuit,  quct_circuit]
synthesis_method_time = [qsd_time,  quct_time]
for res, tim in zip(synthesis_method_result, synthesis_method_time):
    print(f"#gate: {res.n_gates}, #two_qubit_gate: {res.num_two_qubit_gate}, depth: {res.depth}, time: {tim} \n")

#gate: 897, #two_qubit_gate: 379, depth: 815, time: 0.48670148849487305 

#gate: 111, #two_qubit_gate: 35, depth: 54, time: 309.58478021621704 

